In [69]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

#hyper parameters
block_size=8
batch_size=4
max_iters=10000
learning_rate=3e-4
eval_iters=250
dropout = 0.2

cuda


In [70]:
with open("wizard_of_oz.txt", 'r', encoding="utf-8") as f:
    text=f.read()
# creating an array of all the characters in text file
chars=sorted(set(text))   
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


# tokenizing the character

In [71]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda l:"".join([int_to_string[i] for i in l])

In [72]:
#to encode a word we use the following 
print(encode("hellow"))
encoded_word=encode("hellow")
decode_word=decode(encoded_word)
print(f"the encoded word is: {encoded_word}")
print(f"the decoded word is: {decode_word}")

[61, 58, 65, 65, 68, 76]
the encoded word is: [61, 58, 65, 65, 68, 76]
the decoded word is: hellow


In [73]:
# encoding text data to a tourch tensor
data=torch.tensor(encode(text), dtype=torch.long)
print(data[:50])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25])


In [74]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 54, 55, 72, 74, 71, 57,  9],
        [72,  1, 66, 54, 78,  1, 60, 71],
        [78,  1, 61, 54, 57,  1, 56, 68],
        [ 1, 76, 62, 73, 61, 62, 67,  1]], device='cuda:0')
targets:
tensor([[54, 55, 72, 74, 71, 57,  9,  3],
        [ 1, 66, 54, 78,  1, 60, 71, 68],
        [ 1, 61, 54, 57,  1, 56, 68, 66],
        [76, 62, 73, 61, 62, 67,  1, 73]], device='cuda:0')


In [75]:
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when the input is: {context} target is {target}")

when the input is: tensor([80]) target is 28
when the input is: tensor([80, 28]) target is 39
when the input is: tensor([80, 28, 39]) target is 42
when the input is: tensor([80, 28, 39, 42]) target is 39
when the input is: tensor([80, 28, 39, 42, 39]) target is 44
when the input is: tensor([80, 28, 39, 42, 39, 44]) target is 32
when the input is: tensor([80, 28, 39, 42, 39, 44, 32]) target is 49
when the input is: tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is 1


In [81]:
# to make sure pytorch does not use gradients for computation, this is done to reduce the computation time
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ["train","val"]:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y=get_batch(split)
            logits, loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [77]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #defining an embedding vector based on the vacab_size
        self.token_embeddeing_table=nn.Embedding(vocab_size, vocab_size)

    """ 
    below fucntion is used to compute the forward pass of the model and 
    compute the loss for a particular 
    """
    def forward(self, index, targets=None):
        #accessing the embedding vector for a paticular index token 
        logits = self.token_embeddeing_table(index)

        if targets is None:
            loss=None
        else:
            #we unpack the dimension using the .shape
            batch, time, channel=logits.shape
            #we pack the dimension using the .view, we flattenr the batch and time into single dimension to compute loss 
            logits=logits.view(batch*time, channel)
            # we flattern the target vector to get loss
            targets=targets.view(batch*time)
            loss=F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # the index form is (batch, time ) in refrance to the current context
        for _ in range(max_new_tokens):
            #getting the pridition 
            logits, loss=self.forward(index)
            
            #getting the last time step
            logits=logits[:, -1, :] # to make it (batch,  channel)

            #applying softmax to the probablity distribution
            prob=F.softmax(logits, dim=-1) 

            #getting a sample from the distribution
            index_next= torch.multinomial(prob, num_samples=1)

            #appending the generated token to the preexisiting data
            index=torch.cat((index, index_next), dim=-1)
        return index
    

model=BigramLanguageModel(vocab_size)
m=model.to(device)

context=torch.zeros((1,1), dtype=torch.long, device=device)
generated_characters=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_characters)



9?z7:n'N﻿Q:1Y0.AYSTMndLMesz8:cNnGrlJ04PO0Iv﻿*-m?Ue(n9IqPGFJaX:xMtZ﻿0LN4wA-mq,VoNV_ceTI*sQ9dvjs'n'-g4jq8zp:-q
TZ0TMY7vdm)GrTk4y*5(k[ua2w3cdosu﻿d_(Tbs﻿*MuAX !:&oxjq;xhwU4rN2t2-T8dW
Y*lPiRll[FZz8eT'mqWh.Zr ZKGWhpzyy*,'][a;GX0'8absaaTMk﻿"bNdsQ:Eu36
qS9WNv_?I*xwT'CA!]m1_aRFUW
Wh;MPFPOtZN9?R"8t?4nibEVYekZK;81l![6OthdNyheknSrxb﻿tKLDaBNgOptI6L"3vTnoYGYen_;6mmc(WZo5:7﻿0UgEcpz]Ce﻿﻿ qZ"H"3vC0chNn2*fX?Jn0"do2Cp'ZuAdt9W;2x6O-D((A.!45J3 [_Z﻿E [Da4fz[8An2?notxb1ad)EG4S﻿bh﻿;vG[:Xr-mb4w ?fTPV WwJdWSTDabKPR;Q0Q0.


In [84]:
# creating a pytorch optimizer
optimizer=torch.optim.AdamW(model.parameters(), lr=learning_rate)


for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses=estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #sampling a batch data
    xb, yb=get_batch("train")

    #evaluating the loss
    logits, loss=model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.6344, val loss 2.6646
step: 250, train loss: 2.5961, val loss 2.6670
step: 500, train loss: 2.6353, val loss 2.6781
step: 750, train loss: 2.5884, val loss 2.6564
step: 1000, train loss: 2.6191, val loss 2.6279
step: 1250, train loss: 2.5768, val loss 2.6708
step: 1500, train loss: 2.5708, val loss 2.6339
step: 1750, train loss: 2.5712, val loss 2.6334
step: 2000, train loss: 2.5965, val loss 2.6359
step: 2250, train loss: 2.5999, val loss 2.6190
step: 2500, train loss: 2.6088, val loss 2.6081
step: 2750, train loss: 2.5698, val loss 2.5684
step: 3000, train loss: 2.5804, val loss 2.5990
step: 3250, train loss: 2.5722, val loss 2.5950
step: 3500, train loss: 2.5701, val loss 2.6033
step: 3750, train loss: 2.5281, val loss 2.5916
step: 4000, train loss: 2.5599, val loss 2.5874
step: 4250, train loss: 2.5815, val loss 2.5810
step: 4500, train loss: 2.5632, val loss 2.5976
step: 4750, train loss: 2.5626, val loss 2.6042
step: 5000, train loss: 2.5424, val loss 2.569

In [85]:
context=torch.zeros((1,1), dtype=torch.long, device=device)
generated_characters=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_characters)


"Gandn my urowhigr  ulk VBereth t thizbly "Is a s?"BLithther d, wad."HAXpino nuleecl.

Jn,' th. thed ored  T?L*6-me Gud t nivxro o s ted M:X0Ncly ben g te bogsshed
agamus otller stheikell Don thed!id tharheacave te ay d, athed Thwabecch
gy ssustarg;lla ealthen:S[6; id ichaivTO(LAnople
w, bes n why  ibe
J6, &13e gllissacce bu t 9Y7. anveeacecono
"Vm d an 1rive o ith f con br nd.

 hes anlanorerelm Pgidoshe."0U-Jnd, bind "He y A2Nn ok tthentherd ad OU*KxCke, thotht woug ere sariot ew,pe arsannced 
